In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [2]:
def detect_and_predict_mask(frame,faceNet,maskNet):
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.4,177.0,123.0))
    faceNet.setInput(blob)
    detections=faceNet.forward()
    faces=[]
    locs=[]
    preds=[]
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype("int")
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX),min(h-1,endY))
            face=frame[startY:endY,startX:endX]
            if face.size==0:
                continue
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            faces.append(face)
            locs.append((startX,startY,endX,endY))
    if len(faces)>0:
        faces=np.array(faces,dtype='float32')
        preds=maskNet.predict(faces,batch_size=32)

    return(locs,preds)
    

In [3]:
#load our face detector model
prototxtPath=r"F:\playing\deep learning proj\deploy.prototxt"
weightsPath=r"F:\playing\deep learning proj\res10_300x300_ssd_iter_140000.caffemodel" 
faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)
maskNet=load_model("cnn_mask.h5")


In [4]:
print("[INFO] starting video stream...")
vs=VideoStream(src=0).start()
while True:
    frame=vs.read()
    frame=imutils.resize(frame,width=800)
    (locs,preds)=detect_and_predict_mask(frame,faceNet,maskNet)
    for (box,pred) in zip(locs,preds):
        if len(pred)==2:
            (mask,withoutMask)=pred
        else :
            withoutMask=pred[0]
            mask=1-withoutMask
        (startX,startY,endX,endY)=box
        
        label="with Mask" if mask>withoutMask else"No Mask"
        color=(0,255,0)if label =="with Mask" else (0,0,255)
        label="{}:{:.2f}%".format(label,max(mask,withoutMask)*100)
        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
    
    cv2.imshow("frame",frame)
    key=cv2.waitKey(1)
    if key==ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
1/1 [==============================] - 0s 59ms/step
